In [1]:
import speech_recognition as sr
import sys
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, activations, models, preprocessing # type: ignore
import os
import yaml
from tensorflow.keras import preprocessing , utils, Model # type: ignore
import chat as c 
import re

In [2]:

def str_to_tokens( sentence : str, maxlen_questions, tokenizer):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] )
    return preprocessing.sequence.pad_sequences( [tokens_list], maxlen=maxlen_questions , padding='post')

In [3]:
tokenizer = Tokenizer()
enc_model, dec_model, maxlen_questions, maxlen_answers = c.create_and_train(tokenizer=tokenizer)
for _ in range(10):
    words = str(c.record_audio())
    states_values = enc_model.predict(str_to_tokens(words, maxlen_questions, tokenizer))
    empty_target_seq = np.zeros( ( 1 , 1 ) ) 
    print(tokenizer.word_index)
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        
        dec_outputs , h , k = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] ) 
        sampled_word = None
        
        for word , index in tokenizer.word_index.items() :
        
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
        
        empty_target_seq = np.zeros( ( 1 , 1 ) ) 
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , k ] 
        
    print( decoded_translation )


VOCAB SIZE : 1894
(564, 22) 22
(564, 74) 74
(564, 74, 1894)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 22)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 74)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 22, 200)   │    378,800 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 22)        │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 74, 200)   │    378,800 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 200),     │    320,800 │ embedding[0][0],  │
│                     │ (None, 200),      │            │ not_equal[0][0]   │
│                     │ (None, 200)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LSTM (LSTM)         │ [(None, 74, 200), │    320,800 │ embedding_1[0][0… │
│                     │ (None, 200),      │            │ lstm[0][1],       │
│                     │ (None, 200)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_decoder       │ (None, 74, 1894)  │    380,694 │ LSTM[0][0]        │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,779,894 (6.79 MB)

 Trainable params: 1,779,894 (6.79 MB)

 Non-trainable params: 0 (0.00 B)

{1: 'end', 2: 'start', 3: 'you', 4: 'a', 5: 'i', 6: 'the', 7: 'is', 8: 'of', 9: 'to', 10: 'what', 11: 'are', 12: 'do', 13: 'not', 14: 'and', 15: 'me', 16: 'it', 17: 'in', 18: 'have', 19: 'that', 20: 'am', 21: 'tell', 22: 'as', 23: 'can', 24: 'get', 25: 'my', 26: 'when', 27: "i'm", 28: 'your', 29: 'how', 30: 'joke', 31: 'like', 32: 'be', 33: 'an', 34: 'feel', 35: 'about', 36: 'computer', 37: 'who', 38: 'or', 39: 'for', 40: 'no', 41: "don't", 42: 'by', 43: 'cross', 44: 'with', 45: 'software', 46: 'on', 47: 'all', 48: 'much', 49: 'think', 50: 'but', 51: 'very', 52: 'which', 53: 'at', 54: 'he', 55: 'any', 56: 'why', 57: 'know', 58: 'was', 59: 'could', 60: 'so', 61: 'one', 62: 'should', 63: 'from', 64: 'make', 65: 'more', 66: 'we', 67: 'robots', 68: 'die', 69: 'will', 70: 'favorite', 71: 'if', 72: 'did', 73: 'stock', 74: 'human', 75: 'say', 76: 'been', 77: 'emotion', 78: 'robot', 79: 'does', 80: 'mad', 81: 'feeling', 82: 'read', 83: 'hal', 84: "that's", 85: 'really', 86: 'right', 87: 'bad',

KeyError: "Exception encountered when calling Functional.call().\n\n\x1b[1m1862703190368\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=('tf.Tensor(shape=(1, 1), dtype=float32)', 'tf.Tensor(shape=(1, 200), dtype=float32)', 'tf.Tensor(shape=(1, 200), dtype=float32)')\n  • training=False\n  • mask=('None', 'None', 'None')"

: 